In [1]:
import sys
sys.path.append('/home/ivan/distribution_connector')
import os

In [2]:
from sklearn import datasets
import numpy as np
import torch
from tqdm import tqdm
from connector_utils import test_models, gather_statistics, test_func
import matplotlib.pyplot as plt

In [3]:
from connector import Connector
# from one_layer_utils import samples, make_dataset, get_model, get_b
from utils import test_model

In [4]:
import models
architecture = getattr(models, "LinearNoBias") #LinearOneLayer LogRegression

import data
loaders, num_classes = data.loaders(
    "CIFAR10",
    "data",
    1024,
    1,
    "VGG",
    True)

model1 = architecture.base(num_classes=10, **architecture.kwargs)
model2 = architecture.base(num_classes=10, **architecture.kwargs)

model1.load_state_dict(torch.load('curves/Linear6NoBias/curve1/checkpoint-100.pt')['model_state'])
model2.load_state_dict(torch.load('curves/Linear6NoBias/curve2/checkpoint-100.pt')['model_state'])


Files already downloaded and verified
You are going to run models on the test set. Are you sure?
Files already downloaded and verified


IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [5]:
model1.cuda();
model1.eval();
test_model(model1, loaders, cuda=True)

train results {'nll': 0.08421181473970413, 'loss': 0.08421181473970413, 'accuracy': 97.374}
test results {'nll': 2.1362713802337647, 'loss': 2.1362713802337647, 'accuracy': 60.73}


({'nll': 0.08421181473970413, 'loss': 0.08421181473970413, 'accuracy': 97.374},
 {'nll': 2.1362713802337647, 'loss': 2.1362713802337647, 'accuracy': 60.73})

In [6]:
model2.cuda();
test_model(model2, loaders, cuda=True)

train results {'nll': 0.07797462079167367, 'loss': 0.07797462079167367, 'accuracy': 97.69}
test results {'nll': 2.1764473155975343, 'loss': 2.1764473155975343, 'accuracy': 60.49}


({'nll': 0.07797462079167367, 'loss': 0.07797462079167367, 'accuracy': 97.69},
 {'nll': 2.1764473155975343, 'loss': 2.1764473155975343, 'accuracy': 60.49})

In [7]:
def samples(model):
    p = [list(model.parameters())[i].data.cpu().numpy() for i in range(len(list(model.parameters())))]
    return p

def samples_per_layer(model, bias=True):
    p = samples(model)
    if bias:
        p = [np.hstack([p[i], p[i+1][:, None]]) for i in range(0, len(p), 2)]        
    return p

def samples_butterfly(model, bias=True):
    if bias:
        return None
    else:   
        p = samples_per_layer(model, bias=bias)
        parameters = [np.hstack([p[i], p[i+1].T]) for i in range(0, len(p), 2)]
    return parameters

def get_model(W, architecture, bias=False, per_layer=True):
    model_sampled = architecture.base(num_classes=10, **architecture.kwargs)
    model_samples = np.array(W)  

    if per_layer:
        for parameter, w in zip(model_sampled.parameters(), W):
            parameter.data.copy_(torch.from_numpy(w))
    else:
        for i, parameter in enumerate(model_sampled.parameters()):
            w = W[i//2]
            if i % 2 == 0:
                offset = 0
            N = parameter.data.shape[1]
            w_part = w[:, offset:offset+N]
            offset = N
            if i % 2 == 0:
                parameter.data.copy_(torch.from_numpy(w_part))
            else:
                parameter.data.copy_(torch.from_numpy(w_part.T))
            

    return model_sampled

In [8]:
def transform(x1, x2, E12, E22_inv):
    y1 = x1 - E12 @ E22_inv @ x2
    return y1.T
   
def inv_transform(y1, y2, E12, E22_inv):
    x1 = y1 + E12 @ E22_inv @ y2
    return x1.T

In [9]:
parameters1 = samples_per_layer(model1, bias=False)
parameters2 = samples_per_layer(model2, bias=False)

In [31]:
parameters_res_tr = []
parameters_res = []
E12_list = []
E22_inv_list = []

W1, W2 = parameters1[-1], parameters2[-1]

func = 'arc_connect'
cntr = Connector(W1.T, W2.T)
f = getattr(cntr, func)
p_res = f()[1].T

print('p_res', p_res.shape, parameters1[-1].shape)

parameters_res_tr.insert(0, p_res)
parameters_res.insert(0, p_res)

for i in range(len(parameters1)-2, -1, -1):
    
    if i % 2 == 2:
        print('making transform')
        p1 = np.hstack([parameters1[i], W1.T])
        p2 = np.hstack([parameters2[i], W2.T])
        p = np.concatenate([p1,p2])
        
        len_x2 = len(W1)
        print('len', len_x2)
        
        mu1 = p.mean(0)[:-len_x2]
        mu2 = p.mean(0)[len_x2:]

        print('p', p.shape)

        p = p - p.mean(0)    
        cov = p.T @ p

        E12 = cov[:-len_x2, -len_x2:]
        E22 = cov[-len_x2:, -len_x2:]
        E22_inv = np.linalg.inv(E22)

        
        print(1, parameters1[i].T.shape, W1.shape)
        W1 = transform(parameters1[i].T, W1, E12,  E22_inv)
        W2 = transform(parameters2[i].T, W2, E12,  E22_inv)
        print('W1', W1.shape)
    
    else:
        W1 = parameters1[i]
        W2 = parameters2[i]

    func = 'arc_connect'
    if i % 2 == 1:
        print('column')
        cntr = Connector(W1.T, W2.T)
        f = getattr(cntr, func)
        p_res = f()[1].T
    else:
        print('line')
        cntr = Connector(W1, W2)
        f = getattr(cntr, func)
        p_res = f()[1]
    
    
    print(2, p_res.T.shape, parameters_res[0].shape)
    if i % 2 == 2:
        p_res_tr = inv_transform(p_res.T, parameters_res[0], E12,  E22_inv)
    else:
        p_res_tr = p_res
    print('p_res_tr', p_res_tr.shape, parameters1[i].shape)
    parameters_res.insert(0, p_res)
    parameters_res_tr.insert(0, p_res_tr)
    
#     parameters_res_tr.insert(0, p_res)
    
    print('='*10)
    
model = get_model(parameters_res_tr, architecture, per_layer=True)
model.cuda();
model.eval();
test_model(model, loaders, cuda=True)

    

p_res (10, 1000) (10, 1000)
line
2 (1000, 1000) (10, 1000)
p_res_tr (1000, 1000) (1000, 1000)
making transform
p (2000, 2152)
len 1000
1 (1152, 1000) (1000, 1000)


ValueError: operands could not be broadcast together with shapes (1152,1000) (1152,) 

In [19]:
parameters_res_tr = []
parameters_res = []
E12_list = []
E22_inv_list = []

W1, W2 = parameters1[-1], parameters2[-1]

func = 'arc_connect'
cntr = Connector(W1.T, W2.T)
f = getattr(cntr, func)
p_res = f()[1]

print('p_res', p_res.shape, parameters1[-1].shape)

parameters_res_tr.insert(0, p_res.T)
parameters_res.insert(0, p_res.T)

for i in range(len(parameters1)-2, -1, -1):
    
    p1 = np.hstack([parameters1[i], W1.T])
    p2 = np.hstack([parameters2[i], W2.T])
    p = np.concatenate([p1,p2])
    
    print('p', p.shape)
       
    p = p - p.mean(0)    
    cov = p.T @ p
    len_x2 = len(W1)
    print('len', len_x2)
    E12 = cov[:-len_x2, -len_x2:]
    E22 = cov[-len_x2:, -len_x2:]
    E22_inv = np.linalg.inv(E22)
    
    print(1, parameters1[i].T.shape, W1.shape)
    W1 = transform(parameters1[i].T, W1, E12,  E22_inv)
    W2 = transform(parameters2[i].T, W2, E12,  E22_inv)
    
    print('W1', W1.shape)

#     W1 = parameters1[i]
#     W2 = parameters2[i]
    
    func = 'arc_connect'
    cntr = Connector(W1.T, W2.T)
    f = getattr(cntr, func)
    p_res = f()[1].T
    
    
    print(2, p_res.T.shape, parameters_res[0].shape)
    p_res_tr = inv_transform(p_res.T, parameters_res[0], E12,  E22_inv)
    print('p_res_tr', p_res_tr.shape, parameters1[i].shape)
    parameters_res.insert(0, p_res)
    parameters_res_tr.insert(0, p_res_tr)
    
#     parameters_res_tr.insert(0, p_res)
    
    print('='*10)
    
model = get_model(parameters_res_tr, architecture, per_layer=True)
model.cuda();
model.eval();
test_model(model, loaders, cuda=True)

    

p_res (1000, 10) (10, 1000)
p (2000, 1010)
len 10
1 (1000, 1000) (10, 1000)
W1 (1000, 1000)
2 (1000, 1000) (10, 1000)
p_res_tr (1000, 1000) (1000, 1000)
p (2000, 2152)
len 1000
1 (1152, 1000) (1000, 1000)
W1 (1000, 1152)
2 (1152, 1000) (1000, 1000)
p_res_tr (1000, 1152) (1000, 1152)
p (2304, 7144)
len 1000
1 (6144, 1152) (1000, 1152)
W1 (1152, 6144)
2 (6144, 1152) (1000, 1152)
p_res_tr (1152, 6144) (1152, 6144)
p (12288, 7296)
len 1152
1 (6144, 6144) (1152, 6144)
W1 (6144, 6144)
2 (6144, 6144) (1152, 6144)
p_res_tr (6144, 6144) (6144, 6144)
p (12288, 9216)
len 6144
1 (3072, 6144) (6144, 6144)
W1 (6144, 3072)
2 (3072, 6144) (6144, 6144)
p_res_tr (6144, 3072) (6144, 3072)
train results {'nll': 1.5422670337295532, 'loss': 1.5422670337295532, 'accuracy': 53.252}
test results {'nll': 1.728106175994873, 'loss': 1.728106175994873, 'accuracy': 41.85}


({'nll': 1.5422670337295532, 'loss': 1.5422670337295532, 'accuracy': 53.252},
 {'nll': 1.728106175994873, 'loss': 1.728106175994873, 'accuracy': 41.85})

In [20]:
parameters_res_tr = []
parameters_res = []
E12_list = []
E22_inv_list = []



W1, W2 = parameters1[-1], parameters2[-1]

func = 'arc_connect'
cntr = Connector(W1, W2)
f = getattr(cntr, func)
p_res = f()[1]

print('p_res', p_res.shape, parameters1[-1].shape)

parameters_res_tr.insert(0, p_res)
parameters_res.insert(0, p_res)

for i in range(len(parameters1)-2, -1, -1):
    
    p1 = np.hstack([parameters1[i], W1.T])
    p2 = np.hstack([parameters2[i], W2.T])
    p = np.concatenate([p1,p2])
    
    print('p', p.shape)
       
    p = p - p.mean(0)     
    cov = p.T @ p
    len_x2 = len(W1)
    print('len', len_x2)
    E12 = cov[:-len_x2, -len_x2:]
    E22 = cov[-len_x2:, -len_x2:]
    E22_inv = np.linalg.inv(E22)
    
    print(1, parameters1[i].T.shape, W1.shape)
    W1 = transform(parameters1[i].T, W1, E12,  E22_inv)
    W2 = transform(parameters2[i].T, W2, E12,  E22_inv)
    
    print('W1', W1.shape)

#     W1 = parameters1[i]
#     W2 = parameters2[i]
    
    func = 'arc_connect'
    cntr = Connector(W1, W2)
    f = getattr(cntr, func)
    p_res = f()[1]
    
    
    print(2, p_res.T.shape, parameters_res[0].shape)
    p_res_tr = inv_transform(p_res.T, parameters_res[0], E12,  E22_inv)
    print('p_res_tr', p_res_tr.shape, parameters1[i].shape)
    parameters_res.insert(0, p_res)
    parameters_res_tr.insert(0, p_res_tr)
    
#     parameters_res_tr.insert(0, p_res)
    
    print('='*10)
    

model = get_model(parameters_res_tr, architecture, per_layer=True)
model.cuda();
model.eval();
test_model(model, loaders, cuda=True)

p_res (10, 1000) (10, 1000)
p (2000, 1010)
len 10
1 (1000, 1000) (10, 1000)
W1 (1000, 1000)
2 (1000, 1000) (10, 1000)
p_res_tr (1000, 1000) (1000, 1000)
p (2000, 2152)
len 1000
1 (1152, 1000) (1000, 1000)
W1 (1000, 1152)
2 (1152, 1000) (1000, 1000)
p_res_tr (1000, 1152) (1000, 1152)
p (2304, 7144)
len 1000
1 (6144, 1152) (1000, 1152)
W1 (1152, 6144)
2 (6144, 1152) (1000, 1152)
p_res_tr (1152, 6144) (1152, 6144)
p (12288, 7296)
len 1152
1 (6144, 6144) (1152, 6144)
W1 (6144, 6144)
2 (6144, 6144) (1152, 6144)
p_res_tr (6144, 6144) (6144, 6144)
p (12288, 9216)
len 6144
1 (3072, 6144) (6144, 6144)
W1 (6144, 3072)
2 (3072, 6144) (6144, 6144)
p_res_tr (6144, 3072) (6144, 3072)
train results {'nll': 1.5754851763916016, 'loss': 1.5754851763916016, 'accuracy': 48.882}
test results {'nll': 1.7181262357711793, 'loss': 1.7181262357711793, 'accuracy': 39.94}


({'nll': 1.5754851763916016, 'loss': 1.5754851763916016, 'accuracy': 48.882},
 {'nll': 1.7181262357711793, 'loss': 1.7181262357711793, 'accuracy': 39.94})

In [21]:
parameters_res_tr = []
E12_list = []
E22_inv_list = []


W1, W2 = parameters1[5], parameters2[5]

func = 'arc_connect'
cntr = Connector(W1, W2)
f = getattr(cntr, func)
p_res = f()[1]

print('p_res', p_res.shape, parameters1[5].shape)

parameters_res_tr.insert(0, p_res)

for i in range(len(parameters1)-2, -1, -1):
    
    p1 = np.hstack([parameters1[i], parameters1[i+1].T])
    p2 = np.hstack([parameters2[i], parameters2[i+1].T])
    p = np.concatenate([p1,p2])
    
    print('p', p.shape)
    
    p = p - p.mean(0)    
    cov = p.T @ p
    len_x2 = len(W1)
    print('len', len_x2)
    E12 = cov[:-len_x2, -len_x2:]
    E22 = cov[-len_x2:, -len_x2:]
    E22_inv = np.linalg.inv(E22)
    
    print(1, parameters1[i].T.shape, parameters1[i+1].shape)
    W1 = transform(parameters1[i].T, parameters1[i+1], E12,  E22_inv)
    W2 = transform(parameters2[i].T, parameters2[i+1], E12,  E22_inv)
    
    print('W1', W1.shape)

#     W1 = parameters1[i]
#     W2 = parameters2[i]
    
    func = 'arc_connect'
    cntr = Connector(W1, W2)
    f = getattr(cntr, func)
    p_res = f()[1]
    
    
    print(2, p_res.T.shape, parameters_res_tr[0].shape)
    p_res_tr = inv_transform(p_res.T, parameters_res_tr[0], E12,  E22_inv)
    print('p_res_tr', p_res_tr.shape, parameters1[i].shape)
    parameters_res_tr.insert(0, p_res_tr)
    
#     parameters_res_tr.insert(0, p_res)
    
    print('='*10)
    

model = get_model(parameters_res_tr, architecture, per_layer=True)
model.cuda();
model.eval();
test_model(model, loaders, cuda=True)   

p_res (10, 1000) (10, 1000)
p (2000, 1010)
len 10
1 (1000, 1000) (10, 1000)
W1 (1000, 1000)
2 (1000, 1000) (10, 1000)
p_res_tr (1000, 1000) (1000, 1000)
p (2000, 2152)
len 1000
1 (1152, 1000) (1000, 1000)
W1 (1000, 1152)
2 (1152, 1000) (1000, 1000)
p_res_tr (1000, 1152) (1000, 1152)
p (2304, 7144)
len 1000
1 (6144, 1152) (1000, 1152)
W1 (1152, 6144)
2 (6144, 1152) (1000, 1152)
p_res_tr (1152, 6144) (1152, 6144)
p (12288, 7296)
len 1152
1 (6144, 6144) (1152, 6144)
W1 (6144, 6144)
2 (6144, 6144) (1152, 6144)
p_res_tr (6144, 6144) (6144, 6144)
p (12288, 9216)
len 6144
1 (3072, 6144) (6144, 6144)
W1 (6144, 3072)
2 (3072, 6144) (6144, 6144)
p_res_tr (6144, 3072) (6144, 3072)
train results {'nll': 1.5817261745071411, 'loss': 1.5817261745071411, 'accuracy': 49.38}
test results {'nll': 1.7194777265548706, 'loss': 1.7194777265548706, 'accuracy': 39.61}


({'nll': 1.5817261745071411, 'loss': 1.5817261745071411, 'accuracy': 49.38},
 {'nll': 1.7194777265548706, 'loss': 1.7194777265548706, 'accuracy': 39.61})

In [22]:
parameters_res_tr = []
E12_list = []
E22_inv_list = []


W1, W2 = parameters1[5], parameters2[5]

func = 'arc_connect'
cntr = Connector(W1.T, W2.T)
f = getattr(cntr, func)
p_res = f()[1].T

print('p_res', p_res.shape, parameters1[5].shape)

parameters_res_tr.insert(0, p_res)

for i in range(len(parameters1)-2, -1, -1):
    
    p1 = np.hstack([parameters1[i], parameters1[i+1].T])
    p2 = np.hstack([parameters2[i], parameters2[i+1].T])
    p = np.concatenate([p1,p2])
    
    print('p', p.shape)
    
    p = p - p.mean(0)    
    cov = p.T @ p
    len_x2 = len(W1)
    print('len', len_x2)
    E12 = cov[:-len_x2, -len_x2:]
    E22 = cov[-len_x2:, -len_x2:]
    E22_inv = np.linalg.inv(E22)
    
    print(1, parameters1[i].T.shape, parameters1[i+1].shape)
    W1 = transform(parameters1[i].T, parameters1[i+1], E12,  E22_inv)
    W2 = transform(parameters2[i].T, parameters2[i+1], E12,  E22_inv)
    
    print('W1', W1.shape)

#     W1 = parameters1[i]
#     W2 = parameters2[i]
    
    func = 'arc_connect'
    cntr = Connector(W1.T, W2.T)
    f = getattr(cntr, func)
    p_res = f()[1].T
    
    
    print(2, p_res.T.shape, parameters_res_tr[0].shape)
    p_res_tr = inv_transform(p_res.T, parameters_res_tr[0], E12,  E22_inv)
    print('p_res_tr', p_res_tr.shape, parameters1[i].shape)
    parameters_res_tr.insert(0, p_res_tr)
    
#     parameters_res_tr.insert(0, p_res)
    
    print('='*10)
    

model = get_model(parameters_res_tr, architecture, per_layer=True)
model.cuda();
model.eval();
test_model(model, loaders, cuda=True)   

p_res (10, 1000) (10, 1000)
p (2000, 1010)
len 10
1 (1000, 1000) (10, 1000)
W1 (1000, 1000)
2 (1000, 1000) (10, 1000)
p_res_tr (1000, 1000) (1000, 1000)
p (2000, 2152)
len 1000
1 (1152, 1000) (1000, 1000)
W1 (1000, 1152)
2 (1152, 1000) (1000, 1000)
p_res_tr (1000, 1152) (1000, 1152)
p (2304, 7144)
len 1000
1 (6144, 1152) (1000, 1152)
W1 (1152, 6144)
2 (6144, 1152) (1000, 1152)
p_res_tr (1152, 6144) (1152, 6144)
p (12288, 7296)
len 1152
1 (6144, 6144) (1152, 6144)
W1 (6144, 6144)
2 (6144, 6144) (1152, 6144)
p_res_tr (6144, 6144) (6144, 6144)
p (12288, 9216)
len 6144
1 (3072, 6144) (6144, 6144)
W1 (6144, 3072)
2 (3072, 6144) (6144, 6144)
p_res_tr (6144, 3072) (6144, 3072)
train results {'nll': 1.5446964957809448, 'loss': 1.5446964957809448, 'accuracy': 52.534}
test results {'nll': 1.7275915935516357, 'loss': 1.7275915935516357, 'accuracy': 41.48}


({'nll': 1.5446964957809448, 'loss': 1.5446964957809448, 'accuracy': 52.534},
 {'nll': 1.7275915935516357, 'loss': 1.7275915935516357, 'accuracy': 41.48})

In [34]:
parameters1 = samples_butterfly(model1, bias=False)
parameters2 = samples_butterfly(model2, bias=False)
for func in ['lin_connect', 'arc_connect', 'arc_connect_PCA', 'inverse_connect_PCA', 
             'third_cumulant_connect']:
    parameters_res = []
    print(func)
    for p1, p2 in zip(parameters1, parameters2):
        cntr = Connector(p1, p2)
        f = getattr(cntr, func)
        if 'PCA' in func:
            res = f(K=300)[1]
        elif 'third_cumulant' in func:
            res = f(K=100)[1] 
        else:
            res = f()[1]
        parameters_res.append(res)

    model = get_model(parameters_res, architecture, per_layer=False)
    model.cuda();
    model.eval();
    test_model(model, loaders, cuda=True)

lin_connect
train results {'nll': 2.2460536333465577, 'loss': 2.2460536333465577, 'accuracy': 42.888}
test results {'nll': 2.2627061485290527, 'loss': 2.2627061485290527, 'accuracy': 28.65}
arc_connect
train results {'nll': 1.5427578245544433, 'loss': 1.5427578245544433, 'accuracy': 52.87}
test results {'nll': 1.703853747177124, 'loss': 1.703853747177124, 'accuracy': 42.98}
arc_connect_PCA
train results {'nll': 1.5980174809265137, 'loss': 1.5980174809265137, 'accuracy': 53.506}
test results {'nll': 1.7150205610275269, 'loss': 1.7150205610275269, 'accuracy': 44.71}
inverse_connect_PCA
train results {'nll': 1.6406685150527953, 'loss': 1.6406685150527953, 'accuracy': 48.308}
test results {'nll': 1.723065100479126, 'loss': 1.723065100479126, 'accuracy': 42.94}
third_cumulant_connect
c 6627.364648447813 1000000
c 5374.929170055646 1000000
c 3288.9985601145777 1000000
train results {'nll': 64.98299088745117, 'loss': 64.98299088745117, 'accuracy': 9.332}
test results {'nll': 87.94303913574218

In [35]:
parameters1 = samples_per_layer(model1, bias=False)
parameters2 = samples_per_layer(model2, bias=False)
for func in ['lin_connect', 'arc_connect', 'arc_connect_PCA', 'inverse_connect_PCA', 
             'third_cumulant_connect']:
    parameters_res = []
    print(func)
    for p1, p2 in zip(parameters1, parameters2):
        cntr = Connector(p1, p2)
        f = getattr(cntr, func)
        if 'PCA' in func:
            K = min(300, p1.shape[0], p1.shape[1])
            res = f(K=K)[1]
        elif 'third_cumulant' in func:
            K = min(100, p1.shape[0], p1.shape[1])
            res = f(K=K)[1] 
        else:
            res = f()[1]
        parameters_res.append(res)

    model = get_model(parameters_res, architecture, per_layer=True)
    model.cuda();
    model.eval();
    test_model(model, loaders, cuda=True)

lin_connect
train results {'nll': 2.24579755531311, 'loss': 2.24579755531311, 'accuracy': 42.872}
test results {'nll': 2.2627061485290527, 'loss': 2.2627061485290527, 'accuracy': 28.65}
arc_connect
train results {'nll': 1.5418027646636963, 'loss': 1.5418027646636963, 'accuracy': 48.71}
test results {'nll': 1.703467472076416, 'loss': 1.703467472076416, 'accuracy': 40.46}
arc_connect_PCA
train results {'nll': 1.7254339873886109, 'loss': 1.7254339873886109, 'accuracy': 42.358}
test results {'nll': 1.8118197286605835, 'loss': 1.8118197286605835, 'accuracy': 36.62}
inverse_connect_PCA
train results {'nll': 1.9044497087860108, 'loss': 1.9044497087860108, 'accuracy': 33.078}
test results {'nll': 1.9457817846298218, 'loss': 1.9457817846298218, 'accuracy': 30.09}
third_cumulant_connect
c 4672.197923153689 1000000
c 3994.5456819805795 1000000
c 5522.243840191497 1000000
c 4981.6907038015 1000000
c 3287.6942014080832 1000000
c 30.568694950624682 1000
train results {'nll': 4.9637334761047365, 'los

In [14]:
# columns
parameters1 = samples_per_layer(model1, bias=False)
parameters2 = samples_per_layer(model2, bias=False)
for func in ['arc_connect', 'arc_connect_PCA']:
    parameters_res = []
    print(func)
    for p1, p2 in zip(parameters1, parameters2):
        cntr = Connector(p1.T, p2.T)
        f = getattr(cntr, func)
        if 'PCA' in func:
            K = min(300, p1.shape[0], p1.shape[1])
            res = f(K=K)[1]
        elif 'third_cumulant' in func:
            K = min(100, p1.shape[0], p1.shape[1])
            res = f(K=K)[1] 
        else:
            res = f()[1]
        parameters_res.append(res.T)

    model = get_model(parameters_res, architecture, per_layer=True)
    model.cuda();
    model.eval();
    test_model(model, loaders, cuda=True)

lin_connect
train results {'nll': 2.246079969329834, 'loss': 2.246079969329834, 'accuracy': 42.536}
test results {'nll': 2.2627061485290527, 'loss': 2.2627061485290527, 'accuracy': 28.65}
arc_connect
train results {'nll': 1.5765452394104005, 'loss': 1.5765452394104005, 'accuracy': 55.792}
test results {'nll': 1.7582715087890626, 'loss': 1.7582715087890626, 'accuracy': 43.0}
arc_connect_PCA
train results {'nll': 1.662863650779724, 'loss': 1.662863650779724, 'accuracy': 56.806}
test results {'nll': 1.8125662252426147, 'loss': 1.8125662252426147, 'accuracy': 43.27}
inverse_connect_PCA
train results {'nll': 1.7162093257141113, 'loss': 1.7162093257141113, 'accuracy': 49.704}
test results {'nll': 1.8848320421218872, 'loss': 1.8848320421218872, 'accuracy': 37.2}
third_cumulant_connect


KeyboardInterrupt: 

In [16]:
# one last column and lines
parameters1 = samples_per_layer(model1, bias=False)
parameters2 = samples_per_layer(model2, bias=False)
for func in ['arc_connect', 'arc_connect_PCA']:
    parameters_res = []
    print(func)
    for ind, (p1, p2) in enumerate(zip(parameters1, parameters2)):
        if ind==len(parameters1)-1:
            print('last')
            p1, p2 = p1.T, p2.T
        cntr = Connector(p1, p2)
        f = getattr(cntr, func)
        if 'PCA' in func:
            K = min(300, p1.shape[0], p1.shape[1])
            res = f(K=K)[1]
        elif 'third_cumulant' in func:
            K = min(100, p1.shape[0], p1.shape[1])
            res = f(K=K)[1] 
        else:
            res = f()[1]
            
        if ind==len(parameters1)-1:
            res = res.T
        parameters_res.append(res)

    model = get_model(parameters_res, architecture, per_layer=True)
    model.cuda();
    model.eval();
    test_model(model, loaders, cuda=True)

arc_connect
last
train results {'nll': 1.5290968202972413, 'loss': 1.5290968202972413, 'accuracy': 49.078}
test results {'nll': 1.6847684564590455, 'loss': 1.6847684564590455, 'accuracy': 41.33}
arc_connect_PCA
last
train results {'nll': 1.5577786889648437, 'loss': 1.5577786889648437, 'accuracy': 50.992}
test results {'nll': 1.6830526721954346, 'loss': 1.6830526721954346, 'accuracy': 43.3}


In [17]:
# all columns except first lines
parameters1 = samples_per_layer(model1, bias=False)
parameters2 = samples_per_layer(model2, bias=False)
for func in ['arc_connect', 'arc_connect_PCA']:
    parameters_res = []
    print(func)
    for ind, (p1, p2) in enumerate(zip(parameters1, parameters2)):
        if ind!=0:
            print('last')
            p1, p2 = p1.T, p2.T
        cntr = Connector(p1, p2)
        f = getattr(cntr, func)
        if 'PCA' in func:
            K = min(300, p1.shape[0], p1.shape[1])
            res = f(K=K)[1]
        elif 'third_cumulant' in func:
            K = min(100, p1.shape[0], p1.shape[1])
            res = f(K=K)[1] 
        else:
            res = f()[1]
            
        if ind!=0:
            res = res.T
        parameters_res.append(res)

    model = get_model(parameters_res, architecture, per_layer=True)
    model.cuda();
    model.eval();
    test_model(model, loaders, cuda=True)

arc_connect
last
last
last
last
last
train results {'nll': 1.5699368217849732, 'loss': 1.5699368217849732, 'accuracy': 54.064}
test results {'nll': 1.7383880281448365, 'loss': 1.7383880281448365, 'accuracy': 43.39}
arc_connect_PCA
last
last
last
last
last
train results {'nll': 1.6666190790176392, 'loss': 1.6666190790176392, 'accuracy': 54.108}
test results {'nll': 1.7976619497299193, 'loss': 1.7976619497299193, 'accuracy': 43.48}
